# This file is generally to work out the absorption coefficient $I(\vec{k},\vec{G})$

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import astropy.units as unit
import astropy.constants as const
import itertools
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [2]:
#Define known physical constants
a_nai = 6.642 #Lattice constant, Angstroms = 1e-10m, from Dent paper. 
hbar_kevs = (const.hbar.to(unit.keV*unit.s)).value #hbar, keV*s, from astropy
c_aas = (const.c.to(unit.AA/unit.s)).value #speed of light, angstrom/s (cancels with a), from astropy
me_hca = (const.m_e/((const.hbar/(const.c*unit.AA)).si)).value #Electron mass, in units of... hbar/(c*AA) to cancel with |G| (LMAOOOOO)

In [3]:
# Define unknown constants/crystal info
g_agg = 1e-10 #g_agg (axion-photon-photon) coupling constant; in GeV^(-1) (for photon coalescence)
g_ag = 1e-10 #Axion-photon coupling (Primakoff process), in GeV^(-1).
#Upper bound on g_ag of 2.7e-10 from the Sun, see Di Luzio's review
lmda = (g_agg/(1e-8))**4
phi_0 = 5.95e14 #in cm^(-2) s^(-1)
E_0    = 1.103 #in keV

#Crystal Geometry
diam = 110*unit.mm
hgt = 150*unit.mm 
vol = ((np.pi*(diam/2)**2*hgt).to(unit.AA**3)).value #Crystal volume, in AA^3

vol_cell = 67.71 #In AA^3, from Dent paper

In [4]:
#Define useful functions

def make_hkl(max_h=5,max_k=5,max_l=5, s=False):
    full_arr = list(itertools.product(range(max_h), range(max_k), range(max_l)))
    if s:
        filter_func = lambda triple: (triple[0]%2 == triple[1]%2) & (triple[0]%2 == triple[2]%2) & (triple[0]%2 == 0)
        return np.array(list(filter(filter_func, full_arr)))[1:]
    else:
        filter_func = lambda triple: (triple[0]%2 == triple[1]%2) & (triple[0]%2 == triple[2]%2)
        return np.array(list(filter(filter_func, full_arr)))[1:]

def mod2(vec3):
    '''Returns the dimensionless magnitude of an array of vectors in the (h,k,l) basis. Factor of 2pi/a has to be multiplied manually'''
    return 3*(vec3[:,0]**2 + vec3[:,1]**2 + vec3[:,2]**2) - 2*(vec3[:,0]*vec3[:,1] + vec3[:,1]*vec3[:,2] + vec3[:,2]*vec3[:,0])

def gdotk(hkl_arr):
    '''Returns gHAT dot kHAT for a vector in the hkl basis where kHAT is xHAT'''
    return (-hkl_arr[:,0]+hkl_arr[:,1]+hkl_arr[:,2])/np.sqrt(mod2(hkl_arr))

def w_func(Ea, dVector = False, Delta=1.5, E1=1, E2=6):
    '''Returns array of W(Ea, Delta, E1, E2) where everything is in keV. Infinite Ea is dealt with'''
    w_list = []
    if dVector:
        entangle = np.dstack((Ea, Delta))
        for index in range(entangle.shape[1]):
            E = entangle[0,index,0]
            D = entangle[0,index,1]
            if np.isinf(E):
                w_list.append(0)
            else:
                w = (1/2)*(erf((E-E1)/(np.sqrt(2)*D)) - erf((E-E2)/(np.sqrt(2)*D)))
                w_list.append(w)
    else:
        for E in Ea:
            if np.isinf(E):
                w_list.append(0)
            else:
                w = (1/2)*(erf((E-E1)/(np.sqrt(2)*Delta)) - erf((E-E2)/(np.sqrt(2)*Delta)))
                w_list.append(w)
    return np.array(w_list)

def make_dpde(Energy_array):
    '''Creates dPhi/dE for the parametrised blackbody-like form appearing in older papers. Assumes no axion mass.
    Units of cm^(-2) s^(-1) keV^(-1)'''
    dpdt_list = []
    for E in Energy_array:
        if np.isinf(E):
            dpdt_list.append(0)
        else:
            dpdt_list.append(np.sqrt(lmda)*(phi_0)/(E_0) * (E/E_0)**3/(np.exp(E/E_0)-1))
    return np.array(dpdt_list)

def hkl_to_cart(hkl_array):
    '''Converts from hkl basis to cartesian basis, ignoring factor of 2pi/a'''
    x_axis = - hkl_array[:,0] + hkl_array[:,1] + hkl_array[:,2] 
    y_axis =   hkl_array[:,0] - hkl_array[:,1] + hkl_array[:,2] 
    z_axis =   hkl_array[:,0] + hkl_array[:,1] - hkl_array[:,2] 
    return np.stack((x_axis, y_axis, z_axis), axis=1)

In [5]:
hkl = make_hkl(s=True)
hkl_c = hkl_to_cart(hkl)

In [6]:
g_cart = hkl_c*2*np.pi/a_nai #in AA^(-1)

In [7]:
def one_cube(a):
    '''DEPRECATED. Creates an array of 3d points representing one cell in an FCC lattice'''
    return np.array([(0,0,0),(a,0,0),(0,0,a),(a,0,a),
            (0,a,0),(a,a,0),(0,a,a),(a,a,a),
            (0,a/2,a/2), (a/2,0,a/2), (a/2,a/2,0),
            (a,a/2,a/2), (a/2,a,a/2), (a/2,a/2,a)])

In [8]:
def make_lattice_points(cell_vec = (10,10,10), a = a_nai):
    '''Deprecated for being incorrect and slow'''
    #Number of points is almost exactly O(n^3)
    nx = cell_vec[0]
    ny = cell_vec[1]
    nz = cell_vec[2]    

    all_points = one_cube(a) #Initial cube
    init_list = list(itertools.product(range(nx),range(ny),range(nz)))
    
    for start_vec in init_list:
        extend_table = np.repeat(np.array([start_vec]), 14, axis=0)
        points = np.array(one_cube(a)) + extend_table
        all_points = np.vstack((all_points, points))
    return np.unique(all_points, axis=0)

In [9]:
def lattice_3d_scratch(N_x, N_y, N_z):
    full_grid = np.array(tuple(itertools.product(np.arange(0,N_x-0.5,0.5),
                                                 np.arange(0,N_y-0.5,0.5),
                                                 np.arange(0,N_z-0.5,0.5))))
    return np.array(list(filter(lambda x: sum(x%1)%1 == 0, full_grid)))

In [10]:
d3_lattice = lattice_3d_scratch(5,
                                5,
                                5)
#Takes 1m13s for 100^3

In [11]:
d3_lattice

array([[0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 2.],
       ...,
       [4., 4., 2.],
       [4., 4., 3.],
       [4., 4., 4.]])

In [13]:
def difference_array(lattice):
    output = []
    for vec1 in lattice:
        for vec2 in lattice:
            output.append(vec1 - vec2)
    return np.array(output)
#This essentially forms an antisymmetric tensor: treating it as such could cut down time
#JIT compilation w/ numba 

In [14]:
diffs = difference_array(d3_lattice)

In [23]:
df_lattice = pd.DataFrame(d3_lattice)
df_lattice

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,0.0,2.0
3,0.0,0.0,3.0
4,0.0,0.0,4.0
...,...,...,...
360,4.0,4.0,0.0
361,4.0,4.0,1.0
362,4.0,4.0,2.0
363,4.0,4.0,3.0


In [25]:
fig = px.scatter_3d(x = d3_lattice[:,0],
                    y = d3_lattice[:,1],
                    z = d3_lattice[:,2])
#              range_x = (-0.1,1.1),
#              range_y = (-0.1,1.1),
#              range_z = (-0.1,1.1))

fig.update_traces(marker_size = 5)
fig.show()


In [45]:
fig = go.Figure()

scatter = fig.add_trace(go.Scatter3d(
    x=d3_lattice[:,0],
    y=d3_lattice[:,1],
    z=d3_lattice[:,2],
    mode='markers',
    marker = dict(
        size=5,
        color='blue'
    )
))

scatter_io = fig.add_trace(go.Scatter3d(
    x=io_lattice[:,0],
    y=io_lattice[:,1],
    z=io_lattice[:,2],
    mode='markers',
    marker = dict(
        size=5,
        color='red'
    )
))

fig.update_layout(
    scene=dict(
        xaxis=dict(title='', range=[2.4, 4.1]),  
        yaxis=dict(title='', range=[2.4, 4.1]), 
        zaxis=dict(title='', range=[2.4, 4.1])  
    )
)

fig.show()

In [38]:
io_lattice = d3_lattice + np.array([1/2,1/2,1/2])

In [18]:
fig = px.scatter_3d(x = io_lattice[:,0],
                    y = io_lattice[:,1],
                    z = io_lattice[:,2])
#              range_x = (-0.1,1.1),
#              range_y = (-0.1,1.1),
#              range_z = (-0.1,1.1))
fig.update_traces(marker_size = 5, col=3)
fig.show()

IndexError: list index out of range

With the lattice defined, the next step is to incorporate the absorption coefficient:
$$I(\mathbf{k}, \mathbf{G}) = \sum_{i=1}^N \sum_{j\neq i} e^{-\frac{\mathbf{k}-\mathbf{G}}{|\mathbf{k}-\mathbf{G}|} \cdot \frac{\mathbf{r}_i - \mathbf{r}_j}{2\lambda}}$$
$$= \sum_{i,j=1}^N (1-\delta_{ij}) e^{-\frac{\mathbf{k}-\mathbf{G}}{|\mathbf{k}-\mathbf{G}|} \cdot \frac{\mathbf{r}_i - \mathbf{r}_j}{2\lambda}} = \sum_{i,j=1}^N e^{-\frac{\mathbf{k}-\mathbf{G}}{|\mathbf{k}-\mathbf{G}|} \cdot \frac{\mathbf{r}_i - \mathbf{r}_j}{2\lambda}} - N$$
Major issue is that the exponent is not necessarily negative: should I consider only negative-exp values or take abs value of the exponent?

First implementation can just take $\mathbf{k}-\mathbf{G} \propto \hat{\mathbf{x}}$, so take the first element of each r-vector. Mean free path can be estimated at like 100 $\mu m$.
# Take absolute value in exponential (they literally forgot it)
For a crystal on the order 10cm, $\frac{\mathbf{r}_i - \mathbf{r}_j}{2\lambda}$ is on the order 1000

In [37]:
np.sum(np.exp(-50*np.abs(diffs[diffs[:,0] != 0][:,0])))

3.6441964702110125e-07

In [26]:
len(d3_lattice)

365

In [2]:
unit.yr

NameError: name 'unit' is not defined